### PPT 문서 변환
#### Markdown 형식으로 변환

##### 1. PPT 파일 읽기

In [9]:
SAMPLE_PPT = "data/proposal-documents/수행계획서_v.1.1.pptx"

In [60]:
from pptx import Presentation
from langchain_text_splitters import ( Language, RecursiveCharacterTextSplitter, MarkdownHeaderTextSplitter )
from langchain_community.embeddings import OllamaEmbeddings
from langchain_chroma import Chroma
import pathlib
import os

def pptx_to_markdown(pptx_path):
    presentation = Presentation(pptx_path)
    output_path = f"data/output/{pathlib.Path(pptx_path).stem}.md"
    markdown_content = []
    
    # 각 슬라이드(페이지) 처리
    for slide_number, slide in enumerate(presentation.slides, start=1):
        markdown_content.append(f"\n# 슬라이드 {slide_number}\n")
        
        # 슬라이드의 모든 shape 처리
        for shape in slide.shapes:
            if hasattr(shape, "text") and shape.text.strip():
                # 텍스트가 제목 형식인 경우
                if shape.name.startswith("Title"):
                    markdown_content.append(f"## {shape.text.strip()}\n")
                else:
                    # 일반 텍스트인 경우
                    text_lines = shape.text.strip().split('\n')
                    for line in text_lines:
                        if line.strip():
                            markdown_content.append(f"{line.strip()}\n")
    
    # 결과를 마크다운 파일로 저장
    with open(output_path, 'w', encoding='utf-8') as file:
        file.write("\n".join(markdown_content))
        
def load_pptx_to_markdown_advanced(pptx_path):
    presentation = Presentation(pptx_path)
    # output_path = f"data/output/{pathlib.Path(pptx_path).stem}.md"
    markdown_content = []
    
    for slide_number, slide in enumerate(presentation.slides, start=1):
        markdown_content.append(f"\n# 슬라이드 {slide_number}\n")
        
        # 이미지 저장을 위한 디렉토리 생성
        image_dir = "data/output/images"
        os.makedirs(image_dir, exist_ok=True)
        
        for shape in slide.shapes:
            # 텍스트 처리
            if hasattr(shape, "text") and shape.text.strip():
                if shape.name.startswith("Title"):
                    markdown_content.append(f"## {shape.text.strip()}\n")
                else:
                    # 글머리 기호 처리
                    if shape.text.strip().startswith('•'):
                        text_lines = shape.text.strip().split('\n')
                        for line in text_lines:
                            if line.strip().startswith('•'):
                                markdown_content.append("f* {line.strip()[1:]}\n")
                            else:
                                markdown_content.append("f{line.strip()}\n")
                    else:
                        markdown_content.append(f"{shape.text.strip()}\n")
            elif shape.shape_type == 13:
                # 이미지 처리
                image_name = shape.name.strip().replace(' ', '_')
                image_path = f"{image_dir}/{image_name}.png"
                
                if not os.path.exists(image_path):
                    with open(image_path, 'wb') as f:
                        f.write(shape.image.blob)
                markdown_content.append(f"![image]({image_path})\n")
        
    return "".join(markdown_content), pathlib.Path(pptx_path).name

def split_document(markdown_content, source_name):
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200,
        length_function=len,
        language=Language.MARKDOWN
    )
    return splitter.split_documents(markdown_content)

def split_document_from_markdown(markdown_content, source_name):
    headers_to_split_on = [
        ("#", "Header"),
    ]
    
    splitter = MarkdownHeaderTextSplitter(
        headers_to_split_on=headers_to_split_on
    )
    return splitter.split_text(markdown_content)

"""
Split 된 문서를 벡터 스토어에 저장
문서 Embedding 처리 병행
"""
def store_documents(documents):
    db = Chroma.from_documents(
        documents=documents,
        embedding=OllamaEmbeddings(model="nomic-embed-text"),
        persist_directory="data/vectorstore",
        collection_name="proposal-documents"
    )
    return db



In [61]:
markdown_content, source_name = load_pptx_to_markdown_advanced(SAMPLE_PPT)
documents = split_document_from_markdown(markdown_content, source_name)
db = store_documents(documents)

print(db.get())

{'ids': ['1741ce9b-5c5e-4875-aaf8-7ff98de125de', '1b5fbd39-3c63-4f5a-9c77-97a810665289', 'ae352465-db31-4c7c-9d4a-0474c06bf1c0', '5b76b864-1ff9-4174-b034-82eca51121cf', '7935e054-3930-451c-b679-d5c41ed6fe39', 'c130719b-cbe0-4fbc-a7bf-d76fce7af726', '0ff96d1a-62db-4a15-af63-b262a71fcc0d', '0353a789-85ce-45ec-9dbe-7156e63be303', '0cd4be5d-196a-4407-ba2d-666d16bde29e', '70d7576c-bd79-425d-b4b0-05f85c6b9330', '0fe973ff-d349-48ce-96df-15a3c8eed6fc', '953dab9f-c3b7-43be-8993-3957134549ca', 'c631bd82-052e-413d-bac0-b201925b1635', '73a49465-ff62-46b5-ae5d-9d7f7cb30dac', '74ae09c3-d122-4f69-874a-d4c1d88f3fdb', 'a7379c45-4192-42b2-8ca7-3d99550882f2', 'bd174d1f-73d6-4113-8cb0-2281ea1cc151', '9e2af934-677f-4c02-a40d-00a42f0d11db', '7c10a4d9-89c8-4151-a2dd-2a842085b04e', '071ccad3-139c-48fa-bfdd-e7812c2621e2', 'e3fa1bb5-5160-45d3-9d49-104652a925b7', 'dc31e47d-70bc-44ad-9425-c784f3687a94', 'c1168538-68c3-4af2-8f5b-ac2d432eb235'], 'embeddings': None, 'documents': ['멤버스 BI 고도화\n수행계획서\n2018. 12. 07\nDr

In [62]:
db.similarity_search("AI Agent 구축 제안서를 작성해 주세요")

[Document(id='1741ce9b-5c5e-4875-aaf8-7ff98de125de', metadata={'Header': '슬라이드 1'}, page_content='멤버스 BI 고도화\n수행계획서\n2018. 12. 07\nDraft'),
 Document(id='0ff96d1a-62db-4a15-af63-b262a71fcc0d', metadata={'Header': '슬라이드 7'}, page_content='1. LMOS재구축\nLMOS 재구축 분석 요구사항을 바탕으로 분석 유형별 리포트(대시보드, 정형, 비정형, 부서별, 개인리포트)를 구성하며, 캐시 및 스케줄링 적용, 다양한 방식의 분석 템플릿 제공 등 각 분석 유형별 특성에 적합한 솔루션 기술 요소를 적용하여 리포트를 개발합니다.\n제휴사 분석\n제휴사별 총 회원수 현황\n제휴사별 이용 회원수 현황\n제휴사별 이용속성별 현황\n회원 분석\n점별 신규회원 이용 현황\n휴면회원 현황\n우편번호별 회원 현황\n포인트 분석\n포인트 현황\n적립/사용 현황\n카드 실적\n카드종류/발급구분 현황\n멤버스 카드 발급현황\n무기명 카드 교부 및 등록 현황\n제휴사 분석\nSSO 제휴사 현황\n멤버스 분석\n멤버스 가입 유형 현황\n요일별 멤버스 분포\n회원 분석\n신규회원 현황\n기간별 휴면회원 현황\n제휴사별 탈퇴회원 현황\n수수료 정산 분석\n제휴사 유형별 수수료 현황\n카드 종류별 수수료 현황\n캠페인 분석\n쿠폰 발행 대상 고객 추출\n제휴사별 캠페인 대상 고객 추출\n회원 분석\n멤버스 가입 회원 유형 분석\n회원 유형별 포인트 사용 패턴\n제휴사 분석\n제휴사 트렌드 분석\n시스템관리\n![image](data/output/images/그림_54.png)\n접속 현황, 리포트 실행 현황, 개체 사용 현황 등 시스템 사용 통계 조회용 리포트\n![image](data/output/images/그림_56.png)\n개인리포트\n![image](data/output/images/그림_59.png)\